In [1]:
import pandas as pd

In [2]:
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [3]:
df = pd.read_csv("fake_news_LSTM_train.csv", engine="python", encoding='utf-8', error_bad_lines=False)

Skipping line 5762: unexpected end of data


In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
# Drop NaN vallues
df = df.dropna()

In [6]:
# Getting indepedent features
X = df.drop("label", axis = 1)

In [7]:
# Getting depedent features
y = df["label"]

In [8]:
X.shape

(5039, 4)

In [9]:
y.shape

(5039,)

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.4.1'

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense

In [13]:
# Define the volcabulary size
voc_size = 5000

# One Hot representation

In [14]:
# copy the data first
messages = X.copy()

In [15]:
# We reset the index as we have delted tghe NaN values
messages.reset_index(inplace = True)

In [16]:
# Import some libraries for cleaning the data such as removing semicolon, special character
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
# Data Preprocessing
from nltk.stem.porter import PorterStemmer   # steming part is done using PorterStemmer
ps = PorterStemmer()
corpus = []  # creating a list
for i in range(0, len(messages)):
  print(i)
  review = re.sub("[^a-zA-Z]", " ", messages["title"][i])  # substituting character apart from [^a-zA-Z] with blank space " "
  review = review.lower()  # Lowering the letter so that "Thanks" and "thanks" identified as same
  review = review.split()  # splitting the words

  review = [ps.stem(word) for word in review if not word in stopwords.words("english")] # add stem words which are not present in the stopwords
  review = " ".join(review)  # join the newly found words
  corpus.append(review)

Streaming output truncated to the last 5000 lines.
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
29

In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [20]:
# Convert words in the corpus into One hot encoding
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[3486, 984, 2467, 2453, 1061, 2584, 837, 888, 3065, 1796],
 [4437, 1841, 4119, 4379, 4412, 1221, 2020],
 [4908, 3730, 234, 616],
 [1134, 63, 3253, 3769, 421, 2100],
 [4166, 4412, 2150, 1630, 617, 2944, 4412, 490, 3900, 2784],
 [3659,
  132,
  2530,
  1455,
  4505,
  2642,
  2852,
  2198,
  3035,
  4784,
  657,
  1743,
  4444,
  2433,
  2020],
 [2056, 141, 4578, 2579, 3399, 942, 2198, 3311, 4911, 1620, 1424],
 [2900, 3841, 648, 2413, 2116, 2864, 2642, 2011, 4911, 1620, 1424],
 [104, 1821, 3347, 4984, 2086, 2407, 2988, 4608, 2642, 3310],
 [1810, 4091, 2715, 96, 1440, 1768, 1797, 2883],
 [4822, 2302, 4869, 2878, 4123, 2216, 3570, 3300, 4380, 3188, 4045],
 [3769, 2035, 1061, 2407, 2642, 2116],
 [4188, 481, 1118, 2444, 3400, 1691, 3566, 2713, 4093],
 [235, 4785, 756, 3339, 759, 3846, 4022, 4911, 1620, 1424],
 [2487, 490, 3175, 234, 3454, 4911, 1620, 1424],
 [2813, 2610, 1698, 4577, 1032, 2786, 3356, 266, 385, 139],
 [3917, 3662, 1841],
 [3011, 4207, 898, 1511, 2642, 2914, 416, 2020],
 [362

In [21]:
# In one hot representation all words are not of fix lenght. 
# In order to make same lenght of all words we do "pad sequencing" to the words
sent_lenght = 20
embedded_docs = pad_sequences(onehot_repr, padding = "pre", maxlen= sent_lenght)
print(embedded_docs)

[[   0    0    0 ...  888 3065 1796]
 [   0    0    0 ... 4412 1221 2020]
 [   0    0    0 ... 3730  234  616]
 ...
 [   0    0    0 ... 1508  899 2020]
 [   0    0    0 ... 1429  319 1768]
 [   0    0    0 ... 2335  644 4739]]


In [22]:
len(embedded_docs)

5039

In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3486,
        984, 2467, 2453, 1061, 2584,  837,  888, 3065, 1796], dtype=int32)

# Creating the model

In [25]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
embedded_vector_features = 40  # We are assuming the number of feature considered. It can be 100 or 150 also
model = Sequential()
model.add(Embedding(voc_size, embedded_vector_features, input_length = sent_lenght))
model.add(Dropout(0.3))
model.add(LSTM(100)) # We have added one LSTM layer with 100 Nurons
model.add(Dropout(0.3))
model.add(Dense(1, activation= "sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
# Converting the embedded docs into array
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [27]:
X_final.shape, y_final.shape

((5039, 20), (5039,))

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.33, random_state = 42)

# Model Training

In [29]:
# Training
model.fit(X_train, y_train, validation_data= (X_test, y_test), epochs = 10, batch_size= 64)

Epoch 1/10
53/53 [==============================] - 5s 18ms/step - loss: 0.6438 - accuracy: 0.6400 - val_loss: 0.3855 - val_accuracy: 0.8160
Epoch 2/10
53/53 [==============================] - 1s 10ms/step - loss: 0.3051 - accuracy: 0.8648 - val_loss: 0.2540 - val_accuracy: 0.8960
Epoch 3/10
53/53 [==============================] - 1s 10ms/step - loss: 0.1472 - accuracy: 0.9452 - val_loss: 0.2649 - val_accuracy: 0.9056
Epoch 4/10
53/53 [==============================] - 0s 9ms/step - loss: 0.1176 - accuracy: 0.9669 - val_loss: 0.2607 - val_accuracy: 0.9110
Epoch 5/10
53/53 [==============================] - 1s 10ms/step - loss: 0.0455 - accuracy: 0.9860 - val_loss: 0.2529 - val_accuracy: 0.9050
Epoch 6/10
53/53 [==============================] - 0s 9ms/step - loss: 0.0325 - accuracy: 0.9916 - val_loss: 0.2383 - val_accuracy: 0.9104
Epoch 7/10
53/53 [==============================] - 0s 9ms/step - loss: 0.0336 - accuracy: 0.9935 - val_loss: 0.3402 - val_accuracy: 0.8870
Epoch 8/10
53/53

# Performance Mterics and Accuracy

In [30]:
y_pred = model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [31]:
from sklearn.metrics import confusion_matrix

In [32]:
confusion_matrix(y_test, y_pred)

array([[861,  81],
       [ 93, 628]])

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8953698135898978